In [59]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import re
from nltk.util import ngrams

In [60]:
#########1. load the files

In [61]:
query_df = pd.read_pickle('query_df.pkl')
query_df2 = pd.read_pickle('query_df2.pkl')
df = pd.read_pickle('faq_df.pkl')


original = pd.read_excel('3.12阔腿裤客户评论.xlsx', sheet_name = 'ChatGPT分类结果')
scheme_df = pd.read_excel('3.8最新new_faq.xlsx', sheet_name = 'scheme')


In [62]:
##########2.start analysis

In [63]:
def cosine_similarty(a,b):
    res = np.dot(a,b)/(norm(a)*norm(b))
    return res 

In [64]:
def split_(row, col):
    if pd.isnull(row[col]) == False:
        res = re.split('！|，|。|, ', row[col])
    else:
        res = None
    return res 

In [65]:
def scheme_level(row):
    if pd.isnull(row['level 2']) == True:
        res = row['level 1']
    elif pd.isnull(row['level 3']) == True:
        res = row['level 1']+'-'+row['level 2']
    elif pd.isnull(row['level 4']) == True:
        res = row['level 1']+'-'+row['level 2'] +'-'+row['level 3']      
    else:
        res = row['level 1']+'-'+row['level 2'] +'-'+row['level 3']  +'-'+row['level 4']          
    return res

In [66]:
def range_list(start, end, step):
    res = []
    current_value = start

    while current_value <= end:
        res.append(current_value)
        current_value  = current_value + step
        
    return res

In [67]:
###a.split the 分类结果 into molecular

In [68]:
original = original.rename(columns = {'数量序号':'no'})
original['分类结果_new'] = original.apply(split_, axis=1, col = '分类结果')
original = original.explode('分类结果_new')
original


,no,评论内容,分类结果,分类结果_new
0,1,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,"产品-款式-版型-好, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭配建议",产品-款式-版型-好
0,1,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,"产品-款式-版型-好, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭配建议",产品-面料材质-舒适度-舒适
0,1,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,"产品-款式-版型-好, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭配建议",产品-保暖性-好
0,1,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,"产品-款式-版型-好, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭配建议",产品-款式-搭配建议
1,2,竟然一条裤熬过了一冬天，裤子比较保暖，而且比较显瘦,"产品-保暖性-好, 产品-上身效果-好",产品-保暖性-好
...,...,...,...,...
198,199,垂感很好 【尺码大小】裤长有丢丢长 柔软厚实 裤型很挺括,"产品-合身程度-裤长-偏长, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式...",产品-合身程度-裤长-偏长
198,199,垂感很好 【尺码大小】裤长有丢丢长 柔软厚实 裤型很挺括,"产品-合身程度-裤长-偏长, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式...",产品-面料材质-舒适度-舒适
198,199,垂感很好 【尺码大小】裤长有丢丢长 柔软厚实 裤型很挺括,"产品-合身程度-裤长-偏长, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式...",产品-保暖性-好
198,199,垂感很好 【尺码大小】裤长有丢丢长 柔软厚实 裤型很挺括,"产品-合身程度-裤长-偏长, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式...",产品-款式-版型-好


In [69]:
###b. embedding matching on faq_df and query_df

In [70]:
ans_df = pd.DataFrame()
for m in range(0, len(query_df)):
    

    if query_df['评论内容_new'].iloc[m] != '':
        
        ###huggingface
        ####get the similarty score 

        q = query_df['query_embedding'].iloc[m]
        
    
        ####loop through db
        df["similarities"] = df['faq_embedding'].apply(lambda x: cosine_similarty(x[0], q[0]))   


        cnt_df = df.sort_values("similarities", ascending=False)

        #####set top5 count
        cntcnt_df = cnt_df.head(5)
        min_score = cntcnt_df['similarities'].iloc[4]
        max_score = cntcnt_df['similarities'].iloc[0]
        
        group_df = cntcnt_df.groupby('分类结果')['faq_new'].count().reset_index(name='cnt')
        group_df = group_df.sort_values("cnt", ascending=False)
        res = group_df.head(1)['分类结果'].iloc[0]
        ratio_score = group_df.head(1)['cnt'].iloc[0] / 5

        ans_df.at[m,'review_no'] = query_df['评论数序号'].iloc[m]
        ans_df.at[m,'no'] = query_df['no'].iloc[m]
        ans_df.at[m,'query_long'] = query_df['评论内容'].iloc[m]
        ans_df.at[m,'ngram'] = 1
        ans_df.at[m,'query'] = query_df['评论内容_new'].iloc[m]
        ans_df.at[m, 'predict'] = res    
        ans_df.at[m, 'minscore'] = min_score  
        ans_df.at[m, 'maxscore'] = max_score          
        ans_df.at[m, 'ratio'] = ratio_score  

        
for n in range(0, len(query_df2)):
    

    if query_df2['评论内容_new2'].iloc[n] != '':
        
        ###huggingface
        ####get the similarty score 

        q = query_df2['query_embedding'].iloc[n]
        
    
        ####loop through db
        df["similarities"] = df['faq_embedding'].apply(lambda x: cosine_similarty(x[0], q[0]))   


        cnt_df = df.sort_values("similarities", ascending=False)

        #####set top5 count & find the max ratio
        cntcnt_df = cnt_df.head(5)
        min_score = cntcnt_df['similarities'].iloc[4]
        max_score = cntcnt_df['similarities'].iloc[0]
        
        group_df = cntcnt_df.groupby('分类结果')['faq_new'].count().reset_index(name='cnt')
        group_df = group_df.sort_values("cnt", ascending=False)
        res = group_df.head(1)['分类结果'].iloc[0]
        ratio_score = group_df.head(1)['cnt'].iloc[0] / 5

        
        ans_df.at[m+n,'review_no'] = query_df2['评论数序号'].iloc[n]
        ans_df.at[m+n,'no'] = query_df2['no'].iloc[n]
        ans_df.at[m+n,'query_long'] = query_df2['评论内容'].iloc[n]
        ans_df.at[m+n,'ngram'] = 2
        ans_df.at[m+n,'query'] = query_df2['评论内容_new2'].iloc[n]
        ans_df.at[m+n, 'predict'] = res    
        ans_df.at[m+n, 'minscore'] = min_score  
        ans_df.at[m+n, 'maxscore'] = max_score          
        ans_df.at[m+n, 'ratio'] = ratio_score  




In [71]:
####c. calcualte accuracy

In [72]:
############ parameters <---------------------
############ set the range here 
min_score_list = range_list(0.76, 0.8, 0.01)
max_score_list = range_list(0.82, 0.84, 0.01)
ratio_list = range_list(0.6, 0.9, 0.2)

############ measure - choice can be error_per_review or condprob_1_1 or condprob_0_0 or overall_accuracy
choice = 'error_per_review'

In [73]:
###build the dataset

In [74]:
scheme_df['level 1'] = scheme_df['level 1'].ffill()
scheme_df['level 2'] = scheme_df.groupby('level 1')['level 2'].ffill()
scheme_df['level 3'] = scheme_df.groupby(['level 1','level 2'])['level 3'].ffill()
scheme_df['level 4'] = scheme_df.groupby(['level 1','level 2','level 3'])['level 4'].ffill()

scheme_df['cat'] = scheme_df.apply(scheme_level,axis=1)
col_names = scheme_df.cat.unique().tolist()
list_ = original['no'].dropna().unique().tolist()

In [75]:
def create_dataset(df, col, fact):
    
    dataset = pd.DataFrame(columns = col_names)
    
    for n in list_:
        tmp_df = df[df['no']==n]
        if len(tmp_df) > 0:
            res_list = tmp_df[col].dropna().unique().tolist()
            for each in col_names:
                if each in res_list:
                    dataset.at[n,each] = 1
                else:
                    dataset.at[n,each] = 0

        dataset.at[n,'no'] = n
        dataset.at[n,'fact'] = fact
    
    dataset = dataset.fillna(0)
    
    return (dataset)
    

In [76]:
def range_list(start, end, step):
    res = []
    current_value = start

    while current_value <= end:
        res.append(current_value)
        current_value  = current_value + step
        
    return res

In [77]:
def cal_error(col_names, true_result, pred_result):
    df = pd.DataFrame()
    for n in range(0, len(true_result)):
        cnt = 0
        for col in col_names:
            
            if true_result[col].iloc[n] != pred_result[col].iloc[n]:
                cnt = cnt + 1
                
        df.at[n, 'no'] = true_result['no'].iloc[n]
        df.at[n, 'error'] = cnt
        df.at[n, 'correct'] = len(col_names) - cnt
        df.at[n, 'total'] = len(col_names)
    return (df)

In [78]:
def cal_condprob(col_names, true_result, pred_result, given_val, find_val):
    df = pd.DataFrame()
    for n in range(0, len(true_result)):
        
        no = true_result['no'].iloc[n]
        cnt_pred = 0
        cnt_and = 0
        for col in col_names:
            if pred_result[col].iloc[n] == given_val:
                cnt_pred = cnt_pred + 1
                
            if true_result[col].iloc[n] == find_val and pred_result[col].iloc[n] == given_val:
                cnt_and = cnt_and + 1
        
        if cnt_pred == 0:
            prob = 0
        else:
            prob = cnt_and/cnt_pred

        df.at[n, 'no'] = no
        df.at[n, 'given_val'] = given_val
        df.at[n, 'find_val'] = find_val
        df.at[n, 'total_given_pred'] = cnt_pred
        df.at[n, 'total_satisfy'] = cnt_and
        #df.at[n, 'prob'] = prob
    return (df)

In [79]:
def special_lvl4(df):
    df = df.reset_index()
    for m in range(0, len(df)):
        split_list = df['predict'].iloc[m].split('-')
        if len(split_list) == 2:
            df.at[m,'level 1'] = split_list[0]
            df.at[m,'level 2'] = split_list[1]
        elif len(split_list) == 3:
            df.at[m,'level 1'] = split_list[0]
            df.at[m,'level 2'] = split_list[1]            
            df.at[m,'level 3'] = split_list[2] 
        elif len(split_list) == 4:
            df.at[m,'level 1'] = split_list[0]
            df.at[m,'level 2'] = split_list[1]            
            df.at[m,'level 3'] = split_list[2] 
            df.at[m,'level 4'] = split_list[3] 
    return (df)
            

In [80]:
def match_result(row, choice):
    
    
    if type(row['predict_result']) != list:
        row['predict_result'] = []
    if choice == 1:
        ###both true and pred right
        res = list(set(row['true_result']) & set(row['predict_result']))
        
    elif choice == 2:
        ###only in true, but not in pred
        res = list(set(row['true_result']) - set(row['predict_result']))
        
    elif choice == 3:
        ###only in pred, but not in true
        res = list(set(row['predict_result']) - set(row['true_result']))
    
    return res 
        

In [81]:

def measure_sort(df, choice):
    if choice == 'error_per_review':
        df = df.sort_values(by=['error_per_review','condprob_1_1','overall_accuracy'], ascending = [True, False, False])
    elif choice == 'condprob_1_1':
        df = df.sort_values(by=['condprob_1_1','error_per_review','overall_accuracy'], ascending = [False, True, False])
    elif choice == 'overall_accuracy':
        df = df.sort_values(by=['overall_accuracy','error_per_review','condprob_1_1'], ascending = [False, True, False])
    elif choice == 'condprob_0_0':
        df = df.sort_values(by=['condprob_0_0','condprob_1_1','error_per_review','overall_accuracy'], ascending = [False, False, True, False])      
    return df 

In [82]:
report_df = pd.DataFrame()
k=0
for min_ in min_score_list:
    for max_ in max_score_list:
        for ratio in ratio_list:
            

            ####to get distinct pred for each no and as well clear level4 (no dup lv3 with 4)
            filtered_df = ans_df[(ans_df['minscore']>=min_)&(ans_df['maxscore']>=max_)&(ans_df['ratio']>=ratio)]
            filtered_df = special_lvl4(filtered_df)
            filtered_df = filtered_df.sort_values(by=['no','query_long','predict','ratio','maxscore','minscore'], ascending = [True, False, False, False, False,False])
            filtered_df['rank'] = filtered_df.groupby(['review_no','query_long','predict']).cumcount(ascending=True) + 1
            filtered_df = filtered_df[filtered_df['rank']==1]
            filtered_df = filtered_df.sort_values(by=['no','ratio','maxscore','minscore'], ascending = [True, False, False, False])        
            filtered_df['rank2'] = filtered_df.groupby(['review_no','level 1', 'level 2', 'level 3']).cumcount(ascending=True) + 1
            
            
            ####start comparing
            true_result = create_dataset(original, '分类结果_new', 'True_result')
            pred_result = create_dataset(filtered_df, 'predict', 'Predicted_result')

            result_df = cal_error(col_names, true_result, pred_result)
            result_df2_11 = cal_condprob(col_names, true_result, pred_result, 1,1)
            result_df2_00 = cal_condprob(col_names, true_result, pred_result, 0,0)
            
            
            report_df.at[k, 'min_score'] = min_
            report_df.at[k, 'max_score'] = max_
            report_df.at[k, 'ratio'] = ratio
            
            report_df.at[k, 'overall_accuracy'] = (result_df['correct'].sum())/(len(true_result) * len(col_names))
            report_df.at[k, 'error_per_review'] = (result_df['error'].sum())/(len(true_result))
            

            report_df.at[k, 'condprob_1_1'] = result_df2_11['total_satisfy'].sum()/result_df2_11['total_given_pred'].sum()
            report_df.at[k, 'condprob_0_0'] = (result_df2_00['total_satisfy'].sum()/result_df2_00['total_given_pred'].sum())
            
            k=k+1

In [83]:
report_df_final = measure_sort(report_df, choice = choice )
report_df_final.to_excel('allsets.xlsx')

In [84]:
report_df_final.head(10)
        

,min_score,max_score,ratio,overall_accuracy,error_per_review,condprob_1_1,condprob_0_0
0,0.76,0.82,0.6,0.959634,1.655,0.787004,0.972142
6,0.77,0.82,0.6,0.959390,1.665,0.790441,0.971395
2,0.76,0.83,0.6,0.959268,1.670,0.809430,0.969185
8,0.77,0.83,0.6,0.958902,1.685,0.813253,0.968320
12,0.78,0.82,0.6,0.958293,1.710,0.790170,0.969887
14,0.78,0.83,0.6,0.957927,1.725,0.812757,0.967073
4,0.76,0.84,0.6,0.957317,1.750,0.824295,0.965241
10,0.77,0.84,0.6,0.956951,1.765,0.825991,0.964627
1,0.76,0.82,0.8,0.956707,1.775,0.888889,0.959985
20,0.79,0.83,0.6,0.956220,1.795,0.809829,0.965080


In [85]:
######find the best set (taje the firs)
best_min = report_df_final['min_score'].iloc[0]
best_max = report_df_final['max_score'].iloc[0]
best_ratio = report_df_final['ratio'].iloc[0]


print ('best_min:', best_min)
print ('best_max:', best_max)
print ('best_ratio:', best_ratio)

best_min: 0.76
best_max: 0.82
best_ratio: 0.6


In [86]:
#####display the reports for investigation
filtered_df = ans_df[(ans_df['minscore']>=best_min)&(ans_df['maxscore']>=best_max)&(ans_df['ratio']>=best_ratio)]
filtered_df = special_lvl4(filtered_df)
filtered_df = filtered_df.sort_values(by=['no','query_long','predict','ratio','maxscore','minscore'], ascending = [True, False, False, False, False,False])
filtered_df['rank'] = filtered_df.groupby(['review_no','query_long','predict']).cumcount(ascending=True) + 1
filtered_df = filtered_df[filtered_df['rank']==1]
filtered_df = filtered_df.sort_values(by=['no','ratio','maxscore','minscore'], ascending = [True, False, False, False])        
filtered_df['rank2'] = filtered_df.groupby(['review_no','level 1', 'level 2', 'level 3']).cumcount(ascending=True) + 1
   
    
true_result = create_dataset(original, '分类结果_new', 'True_result')
pred_result = create_dataset(filtered_df, 'predict', 'Predicted_result')

result_df = cal_error(col_names, true_result, pred_result)
result_df = pd.merge(result_df, original.groupby(['no'])['分类结果_new'].apply(list).reset_index(name='true_result'), on ='no', how = 'left')
result_df = pd.merge(result_df, filtered_df.groupby(['no'])['predict'].apply(list).reset_index(name='predict_result'), on ='no', how = 'left')
result_df['correct_matches'] = result_df.apply(match_result, axis=1, choice = 1)   
result_df['true1_pred0'] = result_df.apply(match_result, axis=1, choice = 2)   
result_df['true0_pred1'] = result_df.apply(match_result, axis=1, choice = 3)   


result_df3 = original[['no','评论内容']].drop_duplicates()
result_df2 = filtered_df[['no', 'ngram', 'query']].drop_duplicates()
result_df2['query_n'] = result_df2['query'] + '_' + (result_df2['ngram']).astype(str)
result_df2 = result_df2.groupby(['no'])['query_n'].apply(list).reset_index(name='query_ngram')
result_df = pd.merge(result_df, result_df3, on = 'no', how = 'left')
result_df = pd.merge(result_df, result_df2, on = 'no', how = 'left')


result_df.to_excel('bestset.xlsx')




In [87]:
result_df


,no,error,correct,total,true_result,predict_result,correct_matches,true1_pred0,true0_pred1,评论内容,query_ngram
0,1.0,0.0,41.0,41.0,"[产品-款式-版型-好, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭...","[产品-保暖性-好, 产品-面料材质-舒适度-舒适, 产品-款式-搭配建议, 产品-款式-版...","[产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款式-搭配建议, 产品-款式-版...",[],[],非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,"[冬天穿，御寒功能强_2.0, 非常舒适_1.0, 配大衣和羽绒服都合适_1.0, 版型大气..."
1,2.0,2.0,39.0,41.0,"[产品-保暖性-好, 产品-上身效果-好]",NaN,[],"[产品-保暖性-好, 产品-上身效果-好]",[],竟然一条裤熬过了一冬天，裤子比较保暖，而且比较显瘦,NaN
2,3.0,2.0,39.0,41.0,"[产品-服装风格-休闲, 产品-合身程度-尺码-标准, 产品-合身程度-腰围-合适, 产品-...","[产品-合身程度-尺码-标准, 产品-服装风格-休闲, 产品-综合满意度-是否推荐-会推荐,...","[产品-合身程度-尺码-标准, 产品-综合满意度-是否推荐-会推荐, 产品-服装风格-休闲,...","[产品-保暖性-好, 产品-款式-版型-好]",[],很好，正式休闲均可 【尺码大小】合身，腰身舒服 厚，但挺垂的，推荐,"[【尺码大小】合身_1.0, 很好，正式休闲均可_2.0, 推荐_1.0, 【尺码大小】合身..."
3,4.0,2.0,39.0,41.0,"[产品-合身程度-腰围-偏大, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-综...","[产品-面料材质-舒适度-舒适, 产品-保暖性-好]","[产品-面料材质-舒适度-舒适, 产品-保暖性-好]","[产品-合身程度-腰围-偏大, 产品-综合满意度-是否满意-不满意]",[],【尺码大小】裤子太肥了，面料很舒服，也很保暖，做工粗糙,"[面料很舒服_1.0, 也很保暖_1.0]"
4,5.0,1.0,40.0,41.0,"[产品-面料材质-是否易起球-容易起球, 产品-综合满意度-是否满意-不满意, 产品-价格-贵]","[产品-面料材质-是否易起球-容易起球, 产品-价格-贵, 服务-售后-差, 产品-综合满意...","[产品-面料材质-是否易起球-容易起球, 产品-价格-贵, 产品-综合满意度-是否满意-不满意]",[],[服务-售后-差],严重失望，裤子穿了一星期就开始起球，质量不好，真是对不起这个价格，差评差评差评,"[裤子穿了一星期就开始起球_1.0, 真是对不起这个价格_1.0, 差评差评差评_1.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
195,196.0,2.0,39.0,41.0,"[产品-保暖性-好, 产品-合身程度-腰围-偏大]","[产品-保暖性-好, 产品-合身程度-腰围-偏大, 产品-合身程度-腰围-偏小, 产品-上身...","[产品-合身程度-腰围-偏大, 产品-保暖性-好]",[],"[产品-合身程度-腰围-偏小, 产品-上身效果-好]",质量还可以吧，就是长了点，厚实，适合冬天，腰围有点大，胖胖的也能穿，还可以吧。。。。。,"[厚实，适合冬天_2.0, 腰围有点大，胖胖的也能穿_2.0, 适合冬天，腰围有点大_2.0..."
196,197.0,1.0,40.0,41.0,"[产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-合身程度-尺码-标准, 产品-合...","[产品-合身程度-尺码-标准, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-上...","[产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-合身程度-裤长-偏短, 产品-合...",[],[产品-上身效果-好],轻柔的面料，好有一定厚度，尺码合适，稍短 有型且上身舒服,"[尺码合适_1.0, 轻柔的面料_1.0, 好有一定厚度_1.0, 有型且上身舒服_1.0,..."
197,198.0,0.0,41.0,41.0,"[产品-合身程度-尺码-标准, 产品-面料材质-舒适度-舒适, 产品-款式-版型-好, 产品...","[产品-面料材质-舒适度-舒适, 产品-合身程度-尺码-标准, 产品-服装风格-休闲, 产品...","[产品-面料材质-舒适度-舒适, 产品-合身程度-尺码-标准, 产品-服装风格-休闲, 产品...",[],[],【尺码大小】尺码标准，140穿这个码正合适 面料柔软，裤型好看。 可休闲也可正装,"[面料柔软_1.0, 【尺码大小】尺码标准_1.0, 可休闲也可正装_1.0, 裤型好看，_..."
198,199.0,2.0,39.0,41.0,"[产品-合身程度-裤长-偏长, 产品-面料材质-舒适度-舒适, 产品-保暖性-好, 产品-款...","[产品-合身程度-裤长-偏长, 产品-款式-版型-好]","[产品-款式-版型-好, 产品-合身程度-裤长-偏长]","[产品-面料材质-舒适度-舒适, 产品-保暖性-好]",[],垂感很好 【尺码大小】裤长有丢丢长 柔软厚实 裤型很挺括,"[【尺码大小】裤长有丢丢长_1.0, 柔软厚实 裤型很挺括_1.0]"


In [88]:
#######using choice: condprob_1_1 
choice = 'condprob_1_1'
report_df_final = measure_sort(report_df, choice = choice )
report_df_final.head(10)

,min_score,max_score,ratio,overall_accuracy,error_per_review,condprob_1_1,condprob_0_0
5,0.76,0.84,0.8,0.953537,1.905,0.911043,0.955296
11,0.77,0.84,0.8,0.953049,1.925,0.909938,0.954811
17,0.78,0.84,0.8,0.952439,1.950,0.908517,0.954205
23,0.79,0.84,0.8,0.951220,2.000,0.905537,0.952996
15,0.78,0.83,0.8,0.954634,1.860,0.903790,0.956854
9,0.77,0.83,0.8,0.955366,1.830,0.903134,0.957702
3,0.76,0.83,0.8,0.956098,1.800,0.902507,0.958551
29,0.80,0.84,0.8,0.948293,2.120,0.900356,0.949994
21,0.79,0.83,0.8,0.953049,1.925,0.900000,0.955273
27,0.80,0.83,0.8,0.950000,2.050,0.897010,0.952019
